# XGBoost Native API on Adult (Census) Data

In [1]:
import os
os.cpu_count()

12

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

import time

In [3]:
import sys

sys.path.append('C:/Users/scott/ML/')
from library import data_processing

## 1.  Data Processing

In [4]:
df = pd.read_csv('C:/Users/scott/ML/adult/adult.data', sep = ',\s', header = None)


C:\Users\scott\AppData\Local\Temp\ipykernel_9032\499192434.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('C:/Users/scott/ML/adult/adult.data', sep = ',\s', header = None)


In [5]:
df.shape

(32561, 15)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
test = pd.read_csv('C:/Users/scott/ML/adult/adult_test.csv', sep = ',\s')
test.head()

C:\Users\scott\AppData\Local\Temp\ipykernel_9032\687727845.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv('C:/Users/scott/ML/adult/adult_test.csv', sep = ',\s')


,,,,,,,,,,,,,,|1x3 Cross validator
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [8]:
test.reset_index(inplace = True)

In [9]:
test.shape

(16281, 15)

In [10]:
colnames = ['age', 'workclass', 'fnlwgt', 'educatn', 'edctn_num', 'mrtl_sts', 'occptn', 'rlnshp', 'race', 'sex', 'cgain', 'closs', 'hrsprweek', 'ntv_cntry', 'income']

In [11]:
df.columns = colnames
df.head()

,age,workclass,fnlwgt,educatn,edctn_num,mrtl_sts,occptn,rlnshp,race,sex,cgain,closs,hrsprweek,ntv_cntry,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [12]:
test.columns = colnames
test.head()

,age,workclass,fnlwgt,educatn,edctn_num,mrtl_sts,occptn,rlnshp,race,sex,cgain,closs,hrsprweek,ntv_cntry,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [13]:
df.dtypes

age           int64
workclass    object
fnlwgt        int64
educatn      object
edctn_num     int64
mrtl_sts     object
occptn       object
rlnshp       object
race         object
sex          object
cgain         int64
closs         int64
hrsprweek     int64
ntv_cntry    object
income       object
dtype: object

In [14]:
df.describe()

,age,fnlwgt,edctn_num,cgain,closs,hrsprweek
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [15]:
catlist = ['workclass', 'educatn', 'mrtl_sts', 'occptn', 'rlnshp', 'race', 'sex', 'ntv_cntry', 'income']
for v in catlist:
    print(df[v].value_counts())

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64
HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: educatn, dtype: int64
Married-civ-spouse       14976
Never-married            10683
Divorced                  4443
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse           23
Name: mrtl_sts, dtype: int64
Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales 

In [16]:
for v in catlist:
    print(df[v].unique().tolist())

['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov', 'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked']
['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school', '5th-6th', '10th', '1st-4th', 'Preschool', '12th']
['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated', 'Married-AF-spouse', 'Widowed']
['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners', 'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair', 'Transport-moving', 'Farming-fishing', 'Machine-op-inspct', 'Tech-support', '?', 'Protective-serv', 'Armed-Forces', 'Priv-house-serv']
['Not-in-family', 'Husband', 'Wife', 'Own-child', 'Unmarried', 'Other-relative']
['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other']
['Male', 'Female']
['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico', 'South', 'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany', 'Iran', 'Philipp

In [17]:
df['workclass'].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

In [18]:
for v in catlist:
    print(test[v].unique().tolist())

['Private', 'Local-gov', '?', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked']
['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', 'Prof-school', '7th-8th', 'Bachelors', 'Masters', 'Doctorate', '5th-6th', 'Assoc-voc', '9th', '12th', '1st-4th', 'Preschool']
['Never-married', 'Married-civ-spouse', 'Widowed', 'Divorced', 'Separated', 'Married-spouse-absent', 'Married-AF-spouse']
['Machine-op-inspct', 'Farming-fishing', 'Protective-serv', '?', 'Other-service', 'Prof-specialty', 'Craft-repair', 'Adm-clerical', 'Exec-managerial', 'Tech-support', 'Sales', 'Priv-house-serv', 'Transport-moving', 'Handlers-cleaners', 'Armed-Forces']
['Own-child', 'Husband', 'Not-in-family', 'Unmarried', 'Wife', 'Other-relative']
['Black', 'White', 'Asian-Pac-Islander', 'Other', 'Amer-Indian-Eskimo']
['Male', 'Female']
['United-States', '?', 'Peru', 'Guatemala', 'Mexico', 'Dominican-Republic', 'Ireland', 'Germany', 'Philippines', 'Thailand', 'Haiti', 'El-Salvado

In [19]:
pd.crosstab(df['educatn'],df['edctn_num'])

edctn_num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
educatn,,,,,,,,,,,,,,,,
10th,0,0,0,0,0,933,0,0,0,0,0,0,0,0,0,0
11th,0,0,0,0,0,0,1175,0,0,0,0,0,0,0,0,0
12th,0,0,0,0,0,0,0,433,0,0,0,0,0,0,0,0
1st-4th,0,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5th-6th,0,0,333,0,0,0,0,0,0,0,0,0,0,0,0,0
7th-8th,0,0,0,646,0,0,0,0,0,0,0,0,0,0,0,0
9th,0,0,0,0,514,0,0,0,0,0,0,0,0,0,0,0
Assoc-acdm,0,0,0,0,0,0,0,0,0,0,0,1067,0,0,0,0
Assoc-voc,0,0,0,0,0,0,0,0,0,0,1382,0,0,0,0,0


In [20]:
df['cntry'] = np.where(df['ntv_cntry']== ' United-States', 'USA', 'Non_US')
df['y'] = np.where(df['income']== '>50K', 1, 0)

In [21]:
df['y'].value_counts()

0    24720
1     7841
Name: y, dtype: int64

In [22]:
test['income'].value_counts()

<=50K.    12435
>50K.      3846
Name: income, dtype: int64

In [23]:
test['cntry'] = np.where(test['ntv_cntry']== ' United-States', 'USA', 'Non_US')
test['y'] = np.where(test['income']== '>50K.', 1, 0)

In [24]:
test['y'].value_counts()

0    12435
1     3846
Name: y, dtype: int64

In [25]:
df = df.drop(['educatn', 'ntv_cntry', 'income'], axis = 1)

In [26]:
df.head()

,age,workclass,fnlwgt,edctn_num,mrtl_sts,occptn,rlnshp,race,sex,cgain,closs,hrsprweek,cntry,y
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,Non_US,0
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,Non_US,0
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,Non_US,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,Non_US,0
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Non_US,0


In [27]:
test = test.drop(['educatn', 'ntv_cntry', 'income'], axis = 1)

In [28]:
df.replace('?', np.nan, inplace = True)
test.replace('?', np.nan, inplace = True)

In [29]:
df.isna().sum() * 100 / len(df)

age          0.000000
workclass    5.638647
fnlwgt       0.000000
edctn_num    0.000000
mrtl_sts     0.000000
occptn       5.660146
rlnshp       0.000000
race         0.000000
sex          0.000000
cgain        0.000000
closs        0.000000
hrsprweek    0.000000
cntry        0.000000
y            0.000000
dtype: float64

In [30]:
test.isna().sum() * 100 / len(test)

age          0.000000
workclass    5.914870
fnlwgt       0.000000
edctn_num    0.000000
mrtl_sts     0.000000
occptn       5.933296
rlnshp       0.000000
race         0.000000
sex          0.000000
cgain        0.000000
closs        0.000000
hrsprweek    0.000000
cntry        0.000000
y            0.000000
dtype: float64

In [31]:
# LightGBM can natively handle categorical features. For this we need to cast the cattegorical variables as datatype 'category'

df['sex'] = df['sex'].astype('category')
df['workclass'] = df['workclass'].astype('category')
df['mrtl_sts'] = df['mrtl_sts'].astype('category')
df['occptn'] = df['occptn'].astype('category')
df['rlnshp'] = df['rlnshp'].astype('category')
df['race'] = df['race'].astype('category')
df['cntry'] = df['cntry'].astype('category')

In [32]:
cat_features = ['sex', 'workclass', 'mrtl_sts', 'occptn', 'rlnshp', 'race', 'cntry']

In [33]:
df_train, df_val = data_processing.stratified_sampling_with_weight(data = df, weight = 'fnlwgt', test_frac = 0.3, seed = 2025, stratified_col = ['y'])

In [34]:
df_train['fnlwgt'].sum()/df['fnlwgt'].sum()

0.7000109649629018

## 2. Train LightGBM using native api

In [35]:
columns = df_train.columns.tolist()
columns

['age',
 'workclass',
 'fnlwgt',
 'edctn_num',
 'mrtl_sts',
 'occptn',
 'rlnshp',
 'race',
 'sex',
 'cgain',
 'closs',
 'hrsprweek',
 'cntry',
 'y']

In [36]:
target = 'y'
cv_col = 'cv_fold'
weight = 'fnlwgt'

othervars = [target, cv_col, weight]
predictors = [v for v in columns if v not in othervars]

In [37]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [40]:
#Prepare the data

import xgboost as xgb

train_data = xgb.DMatrix(df_train[predictors], label=df_train[target], weight = df_train[weight], enable_categorical = True)
valid_data = xgb.DMatrix(df_val[predictors], label=df_val[target], weight = df_val[weight], enable_categorical = True)

In [56]:
%%time

#Use optuna to tune hyperparameters

import optuna

def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": "gbtree",
        "tree_method": "hist",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log = True),
        "max_depth": trial.suggest_int("max_depth", 2, 6),
        "max_leaves": trial.suggest_int("num_leaves", 3, 50),
        "n_estimators": trial.suggest_int("n_estimators", 200, 800),
        "verbosity": 0
    }
    
    cv_result = xgb.cv(
        params,
        train_data,
        nfold = 3,
        stratified = True,
        metrics = "auc",
        num_boost_round = 500,
        early_stopping_rounds = 10,
        seed = 42,
        verbose_eval = False
    )
    #extract the best iterationn
    best_auc = cv_result["test-auc-mean"].max()
    best_iter = cv_result["test-auc-mean"].idxmax()
    
    #save it in the user atttribute to call later
    trial.set_user_attr("best_iteration", best_iter)
    
    return best_auc

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 30)

print("Best AUC:", study.best_value)
print("Best Parameters:", study.best_params)

[I 2025-06-29 22:41:20,283] A new study created in memory with name: no-name-a8521d84-fe8d-49df-9ede-67a953761ae2
[I 2025-06-29 22:41:34,130] Trial 0 finished with value: 0.9226919162165764 and parameters: {'learning_rate': 0.02698041346387083, 'max_depth': 2, 'num_leaves': 39, 'n_estimators': 504}. Best is trial 0 with value: 0.9226919162165764.
[I 2025-06-29 22:41:37,614] Trial 1 finished with value: 0.928785861325495 and parameters: {'learning_rate': 0.13521199908240303, 'max_depth': 6, 'num_leaves': 48, 'n_estimators': 716}. Best is trial 1 with value: 0.928785861325495.
[I 2025-06-29 22:41:44,610] Trial 2 finished with value: 0.9297860491537268 and parameters: {'learning_rate': 0.09597189598354575, 'max_depth': 6, 'num_leaves': 14, 'n_estimators': 780}. Best is trial 2 with value: 0.9297860491537268.
[I 2025-06-29 22:41:58,640] Trial 3 finished with value: 0.9258851355706041 and parameters: {'learning_rate': 0.04216287142747652, 'max_depth': 2, 'num_leaves': 50, 'n_estimators': 34

Best AUC: 0.9310712334620176
Best Parameters: {'learning_rate': 0.22240368427439378, 'max_depth': 2, 'num_leaves': 44, 'n_estimators': 545}
CPU times: total: 18min 14s
Wall time: 4min 6s


In [58]:
best_iteration = study.best_trial.user_attrs.get("best_iteration", 100)

In [61]:
#Train a model using the best hyperparameters

best_params = study.best_params.copy()

best_params.update({
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "tree_method": "hist"
})


model = xgb.train(
    best_params,
    train_data,
    evals = [(train_data, "train"), (valid_data, "eval")],
    num_boost_round = best_iteration

)

[0]	train-auc:0.84081	eval-auc:0.83582
[1]	train-auc:0.85227	eval-auc:0.84655
[2]	train-auc:0.86220	eval-auc:0.85669
[3]	train-auc:0.86450	eval-auc:0.85925
[4]	train-auc:0.88111	eval-auc:0.87983
[5]	train-auc:0.89554	eval-auc:0.89158
[6]	train-auc:0.89557	eval-auc:0.89175
[7]	train-auc:0.89965	eval-auc:0.89555
[8]	train-auc:0.90279	eval-auc:0.89968
[9]	train-auc:0.90514	eval-auc:0.90096
[10]	train-auc:0.90588	eval-auc:0.90093
[11]	train-auc:0.90840	eval-auc:0.90431
[12]	train-auc:0.90892	eval-auc:0.90471
[13]	train-auc:0.91027	eval-auc:0.90562
[14]	train-auc:0.91223	eval-auc:0.90797
[15]	train-auc:0.91395	eval-auc:0.90948
[16]	train-auc:0.91481	eval-auc:0.91053
[17]	train-auc:0.91501	eval-auc:0.91083
[18]	train-auc:0.91560	eval-auc:0.91175
[19]	train-auc:0.91617	eval-auc:0.91199
[20]	train-auc:0.91719	eval-auc:0.91296
[21]	train-auc:0.91756	eval-auc:0.91374
[22]	train-auc:0.91771	eval-auc:0.91361
[23]	train-auc:0.91805	eval-auc:0.91363
[24]	train-auc:0.91875	eval-auc:0.91430
[25]	train

In [62]:
best_iter = model.best_iteration
print(f"Best Iteration: {model.best_iteration}")

Best Iteration: 230


#### Test Data

In [145]:
#Prepare tthe test data for testinng the model

In [63]:
test['sex'] = test['sex'].astype('category')
test['workclass'] = test['workclass'].astype('category')
test['mrtl_sts'] = test['mrtl_sts'].astype('category')
test['occptn'] = test['occptn'].astype('category')
test['rlnshp'] = test['rlnshp'].astype('category')
test['race'] = test['race'].astype('category')
test['cntry'] = test['cntry'].astype('category')

In [64]:
test_data = xgb.DMatrix(test[predictors], label=test[target], weight = test[weight], enable_categorical = True)

In [67]:
p1 = model.predict(test_data)

In [68]:
from sklearn.metrics  import roc_auc_score

auc = roc_auc_score(test[target], p1, sample_weight = test[weight])
print(f"AUC: {auc:.2f}")
print(f"Gini: {2*auc - 1:.2f}")

AUC: 0.93
Gini: 0.86
